This file is part of the submission of the Chair for Computer Aided
Medical Procedures, Technische Universität München, Germany to the
Prostate Cancer DREAM Challenge 2015.

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.

__Change the path to were the ARFF files produced by the `DREAM_Prostate_Cancer` notebook are located!__

In [ ]:
base_dir = "."

__Change the path to were the `survial` Python package is located!__

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import glob
from os.path import join, basename
import re

import numpy
import pandas
from pandas.rpy.common import convert_to_r_dataframe, convert_robj
from pandas.core.common import is_categorical_dtype
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects import r

from survival.io import loadarff, writearff

In [ ]:
%load_ext rpy2.ipython
%R options(rf.cores = 4)
_rfsrc = importr("randomForestSRC")
f = ro.Formula("Surv(LKADT_P, DEATH) ~ .")

Ensure that attribute names don't contain any illegal characters such as `-`, `(`, or `)`, which get replaced by `.` in R and column names would not match anymore when retrieving results from R.

In [ ]:
def safe_column_rename(table):
    pat = re.compile("[)(-]")
    new_cols = {}
    for col in table.columns:
        new_cols[col] = pat.subn("_", col)[0]
    table.rename(columns=new_cols, inplace=True)

In [ ]:
studies = ["ASCENT2", "CELGENE", "EFC6546"]

In [ ]:
# here's a nice wrapper to combine original data + imputed data
r_impute = r('''combine.impute <- function(object) {
    if (is.null(object$yvar))
        impData <- object$xvar
    else
        impData <- cbind(object$yvar, object$xvar)

    if (!is.null(object$imputed.indv)) {
        impData[object$imputed.indv, ] <- object$imputed.data
    }
    impData
}''')

In [ ]:
def load(study):
    if study.endswith(".arff"):
        filename = study
    else:
        filename = join(base_dir, "%s.arff" % study)
    data = loadarff(filename).set_index("index")
    safe_column_rename(data)
    return data


def category_to_object(data):
    for col in data.select_dtypes(include=['category']).columns:
        data[col] = data[col].astype(object, copy=False)
    return data

def to_rdata(data):
    data = category_to_object(data)
    if "DEATH" in data.columns:
        data["DEATH"] = data["DEATH"].astype(float)
    rdata = convert_to_r_dataframe(data, strings_as_factors=True)
    return rdata


def impute(data):
    rdata = to_rdata(data.copy())
    rfmodel = _rfsrc.rfsrc(f, data=rdata, proximity=False, nsplit=25, ntree=2000, nimpute=5, importance="permute.ensemble",
                           na_action="na.impute", seed=-121451)
    imp_data = convert_robj(r_impute(rfmodel))

    imp_data["DEATH"] = data["DEATH"]
    return imp_data, rfmodel


def load_and_impute(study):
    data = load(study)
    return impute(data)

In [ ]:
imputed_data = {}
for study in studies:
    print(study)
    imputed_data[study] = load_and_impute(study)

In [ ]:
asc_model = imputed_data["ASCENT2"][1]
celg_model = imputed_data["CELGENE"][1]
ven_model = imputed_data["EFC6546"][1]
%Rpush asc_model celg_model ven_model

In [ ]:
%%R
pdf("ASCENT2.pdf", 10, 20); plot(asc_model, plots.one.page=FALSE); dev.off();
pdf("CELGENE.pdf", 10, 20); plot(celg_model, plots.one.page=FALSE); dev.off();
pdf("EFC6546.pdf", 10, 20); plot(ven_model, plots.one.page=FALSE); dev.off();

In [ ]:
for i1 in range(len(studies)):
    s1 = studies[i1]
    for i2 in range(i1 + 1, len(studies)):
        s2 = studies[i2]
        name = "%s_%s" % (s1, s2)
        print(name)

        data = category_to_object(load(name))
        print("Missing values before: %d" % data.isnull().sum().sum())
        data.fillna(imputed_data[s1][0], inplace=True)
        data.fillna(imputed_data[s2][0], inplace=True)
        n_missing = data.isnull().sum().sum()
        print("Missing values after: %d" % n_missing)
        
        if n_missing > 0:
            imputed_data[name] = impute(data)
        else:
            imputed_data[name] = (data, None)

In [ ]:
name = "_".join(studies)
data = category_to_object(load(name))
print("Missing values before: %d" % data.isnull().sum().sum())
for s in studies:
    data.fillna(imputed_data[s][0], inplace=True)

for key, (df, model) in imputed_data.items():
    if key not in studies:
        data.fillna(df, inplace=True)

n_missing = data.isnull().sum().sum()
print("Missing values after: %d" % n_missing)
if n_missing > 0:
    imputed_data[name] = impute(data)
else:
    imputed_data[name] = (data, None)

In [ ]:
#asc_celg_model = imputed_data["ASCENT2_CELGENE"][1]
#asc_ven_model = imputed_data["ASCENT2_EFC6546"][1]
celg_ven_model = imputed_data["CELGENE_EFC6546"][1]
#all_model = imputed_data["ASCENT2_CELGENE_EFC6546"][1]
%Rpush celg_ven_model

In [ ]:
%%R
#pdf("ASCENT2_CELGENE.pdf", 10, 20); plot(asc_celg_model, plots.one.page=FALSE); dev.off();
#pdf("ASCENT2_EFC6546.pdf", 10, 20); plot(asc_ven_model, plots.one.page=FALSE); dev.off();
pdf("CELGENE_EFC6546.pdf", 10, 20); plot(celg_ven_model, plots.one.page=FALSE); dev.off();
#pdf("ASCENT2_CELGENE_EFC6546.pdf", 10, 20); plot(all_model, plots.one.page=FALSE); dev.off();

In [ ]:
def restore_categorical(imp_data, study):
    if isinstance(study, pandas.DataFrame):
        _data = study
    else:
        _data = load(study)
    cat_columns = _data.select_dtypes(include=["category"]).columns
    for col in cat_columns:
        if not is_categorical_dtype(imp_data[col].dtype):
            rc = _data[col].cat
            imp_data[col] = pandas.Categorical(imp_data[col].astype("object"), categories=rc.categories,
                                               ordered=rc.ordered)
    return imp_data

In [ ]:
drop_columns = ["AGEGRP", "HEIGHTBL", "WEIGHTBL"]
for key, (df, model) in imputed_data.items():
    df = restore_categorical(df, key)
    if "STUDYID" in df.columns:
        df = df.drop("STUDYID", axis=1)
    writearff(df.drop(drop_columns, axis=1), "%s-imputed.arff" % key)

# Impute Test Data

__Change the path to where the ARFF files produced by the code above have been written to!__

In [ ]:
train_base_dir = "."

In [ ]:
from dream_utils import transfer_categories

label_cols = ['DEATH', 'LKADT_P', 'DISCONT', 'ENDTRS_C', 'ENTRT_PC']

In [ ]:
# converting directly to R does not support categorical
# and messes up categories that are not present, therefore
# we have to align categories in R again
r_align_factors = r('''align.factors <- function(train.data, test.data) {
for (col in colnames(train.data)) {
    if (!is.factor(train.data[, col]))
        next

    if (!(col %in% colnames(test.data))) {
        next
    }

    lvl.train <- levels(train.data[, col])
    lvl.test <- levels(test.data[, col])

    if (length(setdiff(lvl.test, lvl.train)) > 0) {
#        cat(paste("***", "test", "--->", col, "--->", "train"))
#        print(setdiff(lvl.test, lvl.train))
        new.lvls <- union(lvl.train, lvl.test)

        levels(train.data[, col]) <- new.lvls
    }

    if (length(setdiff(lvl.train, lvl.test)) > 0) {
#        cat(paste("***", "train", "--->", col, "--->", "test"))
#        print(setdiff(lvl.train, lvl.test))
        levels(test.data[, col]) <- lvl.train
    }
}
return(list(train=train.data, test=test.data))
}''')

In [ ]:
def impute_test(train_data, test_data):
#    for col in test_data:
#        if is_categorical_dtype(test_data[col]):
#            print(col)
#            print(test_data[col].cat.categories);print(train_data[col].cat.categories)
    
    print("%d missing values" % test_data.isnull().sum().sum())
    rtrain = to_rdata(train_data)
    rtest = to_rdata(test_data)
    rdata = r_align_factors(rtrain, rtest)

    rf_model = _rfsrc.rfsrc(f, data=rdata.rx2("train"), proximity=False, nsplit=10, importance="none", seed=-121451)
    rpred = r["predict"](rf_model, newdata=rdata.rx2("test"), **{"na.action": "na.impute", "importance": "none"})
    imp_data = r_impute(rpred)

    return convert_robj(imp_data)

In [ ]:
def drop_missing_in_test(train_all):
    cols = train_all.columns - label_cols
    test_missing = test_data.loc[:, cols].apply(lambda x: pandas.isnull(x).sum())
    test_missing /= test_data.shape[0]
    not_in_test_cols = test_missing[test_missing == 1].index
    print("Drop %d columns" % len(not_in_test_cols))

    c = train_all.columns.isin(not_in_test_cols)
    return train_all.drop(train_all.columns[c], axis=1)

In [ ]:
test_data = load(join(base_dir, "dream_test_all.arff"))

In [ ]:
dat_orig = load("_".join(studies))
discont_not_missing = dat_orig.index[dat_orig.DISCONT.notnull()]

In [ ]:
for key in ["ASCENT2", "CELGENE", "EFC6546", "ASCENT2_CELGENE", "ASCENT2_EFC6546", "CELGENE_EFC6546", "ASCENT2_CELGENE_EFC6546"]:
    print(key)
    filename = "%s-imputed.arff" % key
    dat_train = load(join(train_base_dir, filename))
    idx = dat_train.index.isin(discont_not_missing)

    dat_train = drop_missing_in_test(dat_train)

    cols = dat_train.columns - label_cols
    assert cols.isin(test_data.columns).all()
    print(dat_train.shape)
    writearff(dat_train.drop(['DISCONT', 'ENDTRS_C', 'ENTRT_PC'], axis=1), "train_q1_" + filename)
    writearff(dat_train.drop(['DEATH', 'LKADT_P'], axis=1).loc[idx, :], "train_q2_" + filename)

    dat_test = test_data.loc[:, cols]
    dat_train.drop(['DISCONT', 'ENDTRS_C', 'ENTRT_PC'], axis=1, inplace=True)
    dat_test, updates = transfer_categories(
        dat_train, dat_test)
    assert len(updates) == 0

    imp_data = impute_test(dat_train.copy(), dat_test.copy())
    imp_data_cat = restore_categorical(imp_data, dat_train.drop(['DEATH', 'LKADT_P'], axis=1))

    writearff(imp_data_cat, "test_" + filename)